# Info

Генерация предсказаний для тестового датасета.

# Settings

In [29]:
# Files
GDRIVE_DIR = r'/content/drive/MyDrive/DS/20230314_ke-intern-test/'

DATASET_DIR = GDRIVE_DIR + 'dataset/'

TRAIN_EMB_NPZ = GDRIVE_DIR + 'embeddings_text_train.npz'
VAL_EMB_NPZ = GDRIVE_DIR + 'embeddings_text_val.npz'
TEST_EMB_NPZ = GDRIVE_DIR + 'embeddings_text_test.npz'

CATEGORIES_TSV = GDRIVE_DIR + 'categories.tsv'

TEST_DF = GDRIVE_DIR + 'dataset/test.parquet'

# Huperparameters
TOP_K = 3

# Reproducibility
SEED = 1

# Init

## Imports

In [35]:
import gc
import json

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

import torch

## Definitions

In [4]:
#@title  { form-width: "1px", display-mode: "form" }
#@markdown ```python
#@markdown class LabelEncoder(categories_tsv_path: str)
#@markdown ```


class LabelEncoder:
    def __init__(self, categories_tsv_path: str):
        df = pd.read_csv(categories_tsv_path, sep='\t')
        self.category_ids = df.set_index('label').category_id
        self.category_names = df.set_index('category_id').category_name
        self.labels = df.set_index('category_id').label.astype(np.uint32)

    def get_labels(self, category_ids: list) -> np.array:
        return self.labels[category_ids].values

    def get_names(self, category_ids: list) -> pd.Series:
        return self.category_names[category_ids]

    def get_category_ids(self, labels: list) -> pd.Series:
        return self.category_ids[labels]

In [15]:
label_encoder = LabelEncoder(CATEGORIES_TSV)

# Main

## Loading data

In [5]:
train_emb_npz = np.load(TRAIN_EMB_NPZ)
val_emb_npz = np.load(VAL_EMB_NPZ)
test_emb_npz = np.load(TEST_EMB_NPZ)

train_embs = train_emb_npz['embeddings']
train_labels = train_emb_npz['labels']

val_embs = val_emb_npz['embeddings']
val_labels = val_emb_npz['labels']

train_embs = np.r_[train_embs, val_embs]
train_labels = np.r_[train_labels, val_labels]

test_embs = test_emb_npz['embeddings']
test_product_ids = test_emb_npz['product_ids']

print('train', train_embs.shape, train_labels.shape)
print('test', test_embs.shape, test_product_ids.shape)

train (91120, 768) (91120,)
test (16860, 768) (16860,)


### Search best top_k

In [6]:
%%time
similarity_scores = torch.as_tensor(test_embs @ train_embs.T)
similarity_scores.shape

CPU times: user 46 s, sys: 1.53 s, total: 47.5 s
Wall time: 27.9 s


torch.Size([16860, 91120])

In [9]:
del test_embs, train_embs
gc.collect()

55

In [10]:
%%time
train_labels_pt = torch.as_tensor(train_labels.astype(np.int64))

# kNN
_, pred_indices = torch.topk(similarity_scores, k=TOP_K)  # torch.Size([16860, top_k])
pred_top_k_labels = train_labels_pt[pred_indices]  # torch.Size([16860, top_k])
pred_labels = torch.mode(pred_top_k_labels).values  # torch.Size([16860])

CPU times: user 3.73 s, sys: 9.02 ms, total: 3.74 s
Wall time: 3.83 s


### Пример предсказаний

In [31]:
pred_category_ids = label_encoder.get_category_ids(pred_labels.numpy()).values
pred_category_names = label_encoder.get_names(pred_category_ids).values

pred_category_ids, len(pred_category_ids)

(array([13495, 14922,  2803, ..., 13651,  2740, 11757]), 16860)

In [58]:
df_test = pd.read_parquet(TEST_DF)

srs_pred_category_id = pd.Series(pred_category_ids, index=test_product_ids)
srs_pred_category_name = pd.Series(pred_category_names, index=test_product_ids)

df_test['predicted_category_id'] = df_test.product_id.map(srs_pred_category_id)
df_test['predicted_category_name'] = df_test.product_id.map(srs_pred_category_name)

In [60]:
for i in range(5):
    descr = df_test.text_fields.values[i]
    descr = json.loads(descr)

    pred_category = df_test.predicted_category_name.values[i]
    print('Predicted category:\n ', pred_category)
    print('Title:\n ', descr['title'])
    print('Description:\n ', descr['description'])
    print('-'*80)

Predicted category:
  Товары для дома->Товары для праздников->Новогодние товары->Гирлянды
Title:
  Светодиодная лента Smart led Strip Light, с пультом, 5 метров, USB, Bluetooth
Description:
  <p>Светодиодная лента LED, 5 м, RGB (Цветная) влагостойкая лента с пультом, USB адаптером, и возможностью управления с телефона.</p><p>Скачать приложение можно по ссылке <a href="http://www.qrtransfer.com/MiraclesStar.html">http://www.qrtransfer.com/MiraclesStar.html</a></p><p>Гибкая, яркая, с хорошей световой отдачей и низкой ценой. Лента произведена на основе самоклеящейся печатной платы с прочным клеевым слоем «3М». Может нарезаться по 5 см (кратность - 3 диода) без потери их работоспособности, каждый участок может использоваться отдельно, припаиваться при соблюдении контактов в любые формы.</p><p>Температурный спектр светодиодной ленты нейтральный, что делает ее наиболее универсальной для различных целей, благоприятной для освещения и восприятия человеческим глазом.</p><p>Лента очень проста в 

In [62]:
df_preds = df_test[['product_id', 'predicted_category_id']]
df_preds

,product_id,predicted_category_id
1,1997646,13495
2,927375,14922
3,1921513,2803
4,1668662,12044
5,1467778,13887
...,...,...
24987,1914264,11645
24988,1310569,12357
24989,978095,13651
24992,797547,2740


### Save preds

In [63]:
df_preds.to_parquet(GDRIVE_DIR + 'result.parquet')